In [1]:
import os
import pandas as pd
import numpy as np
import datetime

In [2]:
!find data -type f -name '*.csv'

data/swpc/kpindices-2022.csv
data/swpc/kpindices-2023.csv
data/dscovr/dsc_fc_summed_spectra_2022_v01.csv
data/dscovr/dsc_fc_summed_spectra_2023_v01.csv


In [145]:
x_df = pd.read_csv("./data/dscovr/dsc_fc_summed_spectra_2022_v01.csv", delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
y_df = pd.read_csv("./data/swpc/kpindices-2022.csv", delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = 0)

In [146]:
x_df.describe()

,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
count,524450.000000,524450.000000,524450.000000,511816.000000,511811.000000,511809.000000,511809.000000,511815.000000,511815.000000,511814.000000,...,20115.000000,11191.000000,8416.000000,3971.000000,3859.000000,213.000000,139.000000,44.000000,40.000000,35.000000
mean,0.098939,-0.219225,0.060599,58.218162,9.514369,63.246343,56.563212,74.062121,65.428809,94.261894,...,384.711976,332.966637,388.589832,326.915073,294.150454,390.130451,384.123885,376.326977,403.068593,367.789749
std,3.943064,4.535931,3.667523,38.506281,17.031107,44.696797,45.047853,60.528255,82.509457,114.361904,...,40.774888,48.215582,38.280578,57.801942,85.771695,52.754546,51.583450,69.893480,124.156716,143.007012
min,-16.712300,-26.376500,-23.060000,0.231726,0.231726,0.231726,0.231726,0.231726,0.231726,0.231726,...,214.319000,188.544000,211.826000,205.870000,108.950000,202.247000,112.290000,187.778000,0.231726,89.996200
25%,-3.000140,-3.305092,-1.967988,28.843000,0.231726,32.352500,27.916000,37.301550,25.640400,36.973425,...,365.230000,302.532500,367.203750,274.956000,211.853000,359.223000,352.946500,354.073000,366.691750,241.622000
50%,0.212469,-0.288369,0.034746,51.989300,0.386813,55.570600,46.977600,61.355300,45.470200,68.838050,...,383.495000,326.713000,393.807500,323.581000,309.980000,389.128000,395.347000,383.129500,427.067000,403.663000
75%,3.164427,3.017987,2.039108,84.737900,15.812800,90.033900,80.592700,100.403000,84.142500,114.472000,...,404.069000,363.782500,409.080750,377.611000,367.034500,416.967000,410.484000,411.971250,471.866000,449.118500
max,19.725300,22.834700,27.905900,415.389000,385.676000,493.748000,746.807000,1136.670000,1562.550000,1804.560000,...,662.013000,735.132000,772.122000,521.745000,541.136000,637.731000,568.053000,528.470000,655.892000,646.111000


In [147]:
y_df.head()

,date,3,6,9,12,15,18,21,24
0,2022-04-01,3.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0
1,2022-04-02,4.0,5.0,4.0,2.0,2.0,3.0,4.0,3.0
2,2022-04-03,4.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0
3,2022-04-04,4.0,4.0,3.0,2.0,2.0,2.0,2.0,1.0
4,2022-04-05,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0


We can see that the timestamp is different between data sources and we will need to fix that. Because we want to train on a relative measure of time we will convert the data into training examples with an integer ordinal number.

What other changes will be needed? 

- convert date and time to an ordinal count of seconds since the start time of one example
- convert the kp indices to an integer
- transform dscovr NaNs into 0 (for now - maybe there is a better value to use) 
- filter kp rows that have invalid data (-1s)
- normalize dscovr mag field values
- normalize dscovr solar wind values

The last transform we need to make is to organize the data from dscovr into training examples having a shape of (4320, 53, 2). In axis 0 we have 4,320 minutes per 3 days.
In axis 1 we have the 53 data points for each minute.
In axis 2 we have the ISO date timestamp of the Kp index we are learning and the Kp index we want to learn to predict.

Note that we will generate a new training example for every 3 hour Kp index for each day so we want to cache the pr with evious data and shift it for efficiency.

We need to save our training set to files with random training/valid/test splits. We will split 10% of the examples for validation and save them as 'valid.h5' and the remaining 90% will be our training examples stored as 'train.h5'.which is

We still don't have a held-out test set for evaluating the performance of our model. We will use the 2022 data for training and validation sets and the 2023 data as our test set. Of course, you could come back and change this by combining 2022 and 2023 into one big set then splitting it 3 ways, maybe with an 80/10/10 split which is commonly used.



First we will remove any rows where the timestamp is missing

In [148]:
x_df = x_df.drop(x_df[x_df[1].isna()].index)
print(x_df.shape)

(524450, 54)


Now we will remove any rows where all of the solar wind values are missing (NaN)

In [149]:
x_df = x_df.drop(x_df[x_df.loc[:,4:].isnull().all(1)].index)
print(x_df.shape)

(511911, 54)


Now let's replace any remaining NaNs with 0

In [150]:
print(x_df.shape)
values = {}
for i in range(4, 54):
  values[i] = 0
print(values)
x_df = x_df.fillna(values)
print(x_df.shape)

(511911, 54)
{4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0}
(511911, 54)


In [151]:
x_df.describe()

,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
count,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000,...,511911.000000,511911.000000,511911.000000,511911.000000,511911.00000,511911.000000,511911.000000,511911.000000,511911.000000,511911.000000
mean,0.115983,-0.245809,0.071499,58.207358,9.512510,63.233741,56.551942,74.048232,65.416539,94.244032,...,15.116849,7.279058,6.388556,2.535948,2.21743,0.162329,0.104302,0.032346,0.031495,0.025146
std,3.950377,4.545212,3.676730,38.510873,17.029962,44.701258,45.050440,60.531076,82.506584,114.358428,...,75.182786,49.208948,49.656902,29.129386,26.51012,8.028428,6.385241,3.547122,3.723996,3.256721
min,-16.712300,-26.376500,-23.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-2.981530,-3.328865,-1.954055,28.829400,0.231726,32.337400,27.899950,37.288800,25.625400,36.957850,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.237204,-0.323418,0.042795,51.979700,0.386135,55.553800,46.967800,61.336900,45.463100,68.817200,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.177780,2.994075,2.049095,84.733700,15.809700,90.025650,80.582100,100.397500,84.129800,114.466000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
max,19.725300,22.834700,27.905900,415.389000,385.676000,493.748000,746.807000,1136.670000,1562.550000,1804.560000,...,662.013000,735.132000,772.122000,521.745000,541.13600,637.731000,568.053000,528.470000,655.892000,646.111000


In [152]:
M = y_df.shape[0]*(y_df.shape[1]-1)
print(M, 'Kp indices')
print(4320*x_df.shape[1], 'DSCOVR data points for each Kp index')

2920 Kp indices
233280 DSCOVR data points for each Kp index


In [153]:
x_M = int(x_df.shape[0] / 4320)
print(y_df.shape)

(365, 9)


In [154]:
from sklearn.model_selection import train_test_split

train_ydf, valid_ydf = train_test_split(y_df, test_size=0.1)

In [155]:
train_M = train_ydf.shape[0] * (train_ydf.shape[1]-1)
valid_M = valid_ydf.shape[0] * (valid_ydf.shape[1]-1)
print(train_M, ' training examples and ', valid_M, ' validation examples')

2624  training examples and  296  validation examples


In [156]:
from datetime import timedelta
def get_X(t, indf):
  match = indf.loc[:,0]==t
  if indf[match].shape[0] == 1:
    x_idx = indf[match].index[0]
  else:
    x_idx = -1
    for h in range(24):
      nt = pd.to_datetime(t) - timedelta(hours=h)
      match = indf.loc[:,0]==nt
      if indf[match].shape[0] == 1:
        x_idx = indf[match].index[0]
        break
  return x_idx, x_idx+4320

Check that get_X performs as expected

In [157]:
t = valid_ydf.iloc[0,0]
begin, end = get_X(t, x_df)
print('Start at ',x_df.loc[begin,0])
print('End   at ',x_df.loc[end,0])
print(valid_M / 8)

Start at  2022-09-24 00:00:00
End   at  2022-09-27 00:00:00
37.0


Now step through each validation example and construct one training example with 4320 one minute (3 days) solar wind samples 

In [161]:
valid_X = np.zeros((valid_M, 4320, 53))
valid_Y = np.zeros((valid_M, 1))
for m in range(valid_M):
  y_0 = int(m / 8)
  y_1 = m % 8
  print(m, y_0, y_1, valid_ydf.shape, t)
  t = valid_ydf.iloc[y_0, 0]
  #dt = datetime.datetime.strptime(t, '%Y%m%d')
  #print(t, dt)
  seq = get_X(t, x_df)
  # check for exactly 3 days of data
  print(seq)
  if seq[1] < x_df.shape[0]:
    print(m, 'Start at ',x_df.iloc[seq[0],0], ' for ', x_df.iloc[seq[1],0] - x_df.iloc[seq[0],0])
    if x_df.iloc[seq[1],0] - x_df.iloc[seq[0],0] == '3 days 00:00:00':
      #print(seq[1], x_df.shape[0])
      valid_X[m] = x_df.iloc[seq[0]:seq[1],1:].to_numpy()
      valid_Y[m] = valid_ydf.iloc[y_0, y_1+1]
      break

0 0 0 (37, 9) 2022-09-24 00:00:00
(383040, 387360)
0 Start at  2022-10-01 14:12:00  for  3 days 00:00:00
1 0 1 (37, 9) 2022-09-24 00:00:00
(383040, 387360)
1 Start at  2022-10-01 14:12:00  for  3 days 00:00:00
2 0 2 (37, 9) 2022-09-24 00:00:00
(383040, 387360)
2 Start at  2022-10-01 14:12:00  for  3 days 00:00:00
3 0 3 (37, 9) 2022-09-24 00:00:00
(383040, 387360)
3 Start at  2022-10-01 14:12:00  for  3 days 00:00:00
4 0 4 (37, 9) 2022-09-24 00:00:00
(383040, 387360)
4 Start at  2022-10-01 14:12:00  for  3 days 00:00:00
5 0 5 (37, 9) 2022-09-24 00:00:00
(383040, 387360)
5 Start at  2022-10-01 14:12:00  for  3 days 00:00:00
6 0 6 (37, 9) 2022-09-24 00:00:00
(383040, 387360)
6 Start at  2022-10-01 14:12:00  for  3 days 00:00:00
7 0 7 (37, 9) 2022-09-24 00:00:00
(383040, 387360)
7 Start at  2022-10-01 14:12:00  for  3 days 00:00:00
8 1 0 (37, 9) 2022-09-24 00:00:00
(375840, 380160)
8 Start at  2022-09-26 14:10:00  for  3 days 00:02:00
9 1 1 (37, 9) 2022-09-19 00:00:00
(375840, 380160)
9 St

(416160, 420480)
152 Start at  2022-10-24 14:32:00  for  3 days 03:57:00
153 19 1 (37, 9) 2022-10-17 00:00:00
(416160, 420480)
153 Start at  2022-10-24 14:32:00  for  3 days 03:57:00
154 19 2 (37, 9) 2022-10-17 00:00:00
(416160, 420480)
154 Start at  2022-10-24 14:32:00  for  3 days 03:57:00
155 19 3 (37, 9) 2022-10-17 00:00:00
(416160, 420480)
155 Start at  2022-10-24 14:32:00  for  3 days 03:57:00
156 19 4 (37, 9) 2022-10-17 00:00:00
(416160, 420480)
156 Start at  2022-10-24 14:32:00  for  3 days 03:57:00
157 19 5 (37, 9) 2022-10-17 00:00:00
(416160, 420480)
157 Start at  2022-10-24 14:32:00  for  3 days 03:57:00
158 19 6 (37, 9) 2022-10-17 00:00:00
(416160, 420480)
158 Start at  2022-10-24 14:32:00  for  3 days 03:57:00
159 19 7 (37, 9) 2022-10-17 00:00:00
(416160, 420480)
159 Start at  2022-10-24 14:32:00  for  3 days 03:57:00
160 20 0 (37, 9) 2022-10-17 00:00:00
(243360, 247680)
160 Start at  2022-06-24 12:05:00  for  3 days 00:23:00
161 20 1 (37, 9) 2022-06-19 00:00:00
(243360, 2

In [162]:
train_X = np.zeros((train_M, 4320, 53))
train_Y = np.zeros((train_M, 1))
for m in range(train_M):
  y_0 = int(m / 8)
  y_1 = m % 8
  t = train_ydf.iloc[y_0, 0]
  print(m, y_0, y_1, train_ydf.shape, t)
  #dt = datetime.datetime.strptime(t, '%Y%m%d')
  #print(t, dt)
  seq = get_X(t, x_df)
  # check for exactly 3 days of data
  print(seq)
  if seq[1] < x_df.shape[0]:
    print(m, 'Start at ',x_df.iloc[seq[0],0], ' for ', x_df.iloc[seq[1],0] - x_df.iloc[seq[0],0])
    if x_df.iloc[seq[1],0] - x_df.iloc[seq[0],0] == '3 days 00:00:00':
      #print(seq[1], x_df.shape[0])
      train_X[m] = x_df.iloc[seq[0]:seq[1],1:].to_numpy()
      train_Y[m] = train_ydf.iloc[y_0, y_1+1]
      break

0 0 0 (328, 9) 2022-11-25 00:00:00
(472320, 476640)
0 Start at  2022-12-03 07:53:00  for  3 days 00:00:00
1 0 1 (328, 9) 2022-11-25 00:00:00
(472320, 476640)
1 Start at  2022-12-03 07:53:00  for  3 days 00:00:00
2 0 2 (328, 9) 2022-11-25 00:00:00
(472320, 476640)
2 Start at  2022-12-03 07:53:00  for  3 days 00:00:00
3 0 3 (328, 9) 2022-11-25 00:00:00
(472320, 476640)
3 Start at  2022-12-03 07:53:00  for  3 days 00:00:00
4 0 4 (328, 9) 2022-11-25 00:00:00
(472320, 476640)
4 Start at  2022-12-03 07:53:00  for  3 days 00:00:00
5 0 5 (328, 9) 2022-11-25 00:00:00
(472320, 476640)
5 Start at  2022-12-03 07:53:00  for  3 days 00:00:00
6 0 6 (328, 9) 2022-11-25 00:00:00
(472320, 476640)
6 Start at  2022-12-03 07:53:00  for  3 days 00:00:00
7 0 7 (328, 9) 2022-11-25 00:00:00
(472320, 476640)
7 Start at  2022-12-03 07:53:00  for  3 days 00:00:00
8 1 0 (328, 9) 2022-10-24 00:00:00
(426240, 430560)
8 Start at  2022-10-31 18:52:00  for  3 days 00:03:00
9 1 1 (328, 9) 2022-10-24 00:00:00
(426240, 43

(452160, 456480)
153 Start at  2022-11-19 01:01:00  for  3 days 00:00:00
154 19 2 (328, 9) 2022-11-11 00:00:00
(452160, 456480)
154 Start at  2022-11-19 01:01:00  for  3 days 00:00:00
155 19 3 (328, 9) 2022-11-11 00:00:00
(452160, 456480)
155 Start at  2022-11-19 01:01:00  for  3 days 00:00:00
156 19 4 (328, 9) 2022-11-11 00:00:00
(452160, 456480)
156 Start at  2022-11-19 01:01:00  for  3 days 00:00:00
157 19 5 (328, 9) 2022-11-11 00:00:00
(452160, 456480)
157 Start at  2022-11-19 01:01:00  for  3 days 00:00:00
158 19 6 (328, 9) 2022-11-11 00:00:00
(452160, 456480)
158 Start at  2022-11-19 01:01:00  for  3 days 00:00:00
159 19 7 (328, 9) 2022-11-11 00:00:00
(452160, 456480)
159 Start at  2022-11-19 01:01:00  for  3 days 00:00:00
160 20 0 (328, 9) 2022-05-10 00:00:00
(185760, 190080)
160 Start at  2022-05-15 00:01:00  for  3 days 00:00:00
161 20 1 (328, 9) 2022-05-10 00:00:00
(185760, 190080)
161 Start at  2022-05-15 00:01:00  for  3 days 00:00:00
162 20 2 (328, 9) 2022-05-10 00:00:00
(

(5760, 10080)
308 Start at  2022-01-05 09:27:00  for  3 days 09:42:00
309 38 5 (328, 9) 2022-01-05 00:00:00
(5760, 10080)
309 Start at  2022-01-05 09:27:00  for  3 days 09:42:00
310 38 6 (328, 9) 2022-01-05 00:00:00
(5760, 10080)
310 Start at  2022-01-05 09:27:00  for  3 days 09:42:00
311 38 7 (328, 9) 2022-01-05 00:00:00
(5760, 10080)
311 Start at  2022-01-05 09:27:00  for  3 days 09:42:00
312 39 0 (328, 9) 2022-03-05 00:00:00
(90720, 95040)
312 Start at  2022-03-08 21:54:00  for  3 days 00:01:00
313 39 1 (328, 9) 2022-03-05 00:00:00
(90720, 95040)
313 Start at  2022-03-08 21:54:00  for  3 days 00:01:00
314 39 2 (328, 9) 2022-03-05 00:00:00
(90720, 95040)
314 Start at  2022-03-08 21:54:00  for  3 days 00:01:00
315 39 3 (328, 9) 2022-03-05 00:00:00
(90720, 95040)
315 Start at  2022-03-08 21:54:00  for  3 days 00:01:00
316 39 4 (328, 9) 2022-03-05 00:00:00
(90720, 95040)
316 Start at  2022-03-08 21:54:00  for  3 days 00:01:00
317 39 5 (328, 9) 2022-03-05 00:00:00
(90720, 95040)
317 Star

(408960, 413280)
461 Start at  2022-10-19 14:29:00  for  3 days 00:03:00
462 57 6 (328, 9) 2022-10-12 00:00:00
(408960, 413280)
462 Start at  2022-10-19 14:29:00  for  3 days 00:03:00
463 57 7 (328, 9) 2022-10-12 00:00:00
(408960, 413280)
463 Start at  2022-10-19 14:29:00  for  3 days 00:03:00
464 58 0 (328, 9) 2022-12-04 00:00:00
(485280, 489600)
464 Start at  2022-12-12 07:59:00  for  3 days 01:08:00
465 58 1 (328, 9) 2022-12-04 00:00:00
(485280, 489600)
465 Start at  2022-12-12 07:59:00  for  3 days 01:08:00
466 58 2 (328, 9) 2022-12-04 00:00:00
(485280, 489600)
466 Start at  2022-12-12 07:59:00  for  3 days 01:08:00
467 58 3 (328, 9) 2022-12-04 00:00:00
(485280, 489600)
467 Start at  2022-12-12 07:59:00  for  3 days 01:08:00
468 58 4 (328, 9) 2022-12-04 00:00:00
(485280, 489600)
468 Start at  2022-12-12 07:59:00  for  3 days 01:08:00
469 58 5 (328, 9) 2022-12-04 00:00:00
(485280, 489600)
469 Start at  2022-12-12 07:59:00  for  3 days 01:08:00
470 58 6 (328, 9) 2022-12-04 00:00:00
(

(289440, 293760)
614 Start at  2022-07-28 04:47:00  for  3 days 00:00:00
615 76 7 (328, 9) 2022-07-21 00:00:00
(289440, 293760)
615 Start at  2022-07-28 04:47:00  for  3 days 00:00:00
616 77 0 (328, 9) 2022-02-06 00:00:00
(51840, 56160)
616 Start at  2022-02-09 05:52:00  for  3 days 00:13:00
617 77 1 (328, 9) 2022-02-06 00:00:00
(51840, 56160)
617 Start at  2022-02-09 05:52:00  for  3 days 00:13:00
618 77 2 (328, 9) 2022-02-06 00:00:00
(51840, 56160)
618 Start at  2022-02-09 05:52:00  for  3 days 00:13:00
619 77 3 (328, 9) 2022-02-06 00:00:00
(51840, 56160)
619 Start at  2022-02-09 05:52:00  for  3 days 00:13:00
620 77 4 (328, 9) 2022-02-06 00:00:00
(51840, 56160)
620 Start at  2022-02-09 05:52:00  for  3 days 00:13:00
621 77 5 (328, 9) 2022-02-06 00:00:00
(51840, 56160)
621 Start at  2022-02-09 05:52:00  for  3 days 00:13:00
622 77 6 (328, 9) 2022-02-06 00:00:00
(51840, 56160)
622 Start at  2022-02-09 05:52:00  for  3 days 00:13:00
623 77 7 (328, 9) 2022-02-06 00:00:00
(51840, 56160)


...

(67680, 72000)
2065 Start at  2022-02-20 06:05:00  for  3 days 00:00:00
2066 258 2 (328, 9) 2022-02-17 00:00:00
(67680, 72000)
2066 Start at  2022-02-20 06:05:00  for  3 days 00:00:00
2067 258 3 (328, 9) 2022-02-17 00:00:00
(67680, 72000)
2067 Start at  2022-02-20 06:05:00  for  3 days 00:00:00
2068 258 4 (328, 9) 2022-02-17 00:00:00
(67680, 72000)
2068 Start at  2022-02-20 06:05:00  for  3 days 00:00:00
2069 258 5 (328, 9) 2022-02-17 00:00:00
(67680, 72000)
2069 Start at  2022-02-20 06:05:00  for  3 days 00:00:00
2070 258 6 (328, 9) 2022-02-17 00:00:00
(67680, 72000)
2070 Start at  2022-02-20 06:05:00  for  3 days 00:00:00
2071 258 7 (328, 9) 2022-02-17 00:00:00
(67680, 72000)
2071 Start at  2022-02-20 06:05:00  for  3 days 00:00:00
2072 259 0 (328, 9) 2022-02-13 00:00:00
(61920, 66240)
2072 Start at  2022-02-16 06:05:00  for  3 days 00:00:00
2073 259 1 (328, 9) 2022-02-13 00:00:00
(61920, 66240)
2073 Start at  2022-02-16 06:05:00  for  3 days 00:00:00
2074 259 2 (328, 9) 2022-02-13 0

(423360, 427680)
2200 Start at  2022-10-29 18:52:00  for  3 days 00:02:00
2201 275 1 (328, 9) 2022-10-22 00:00:00
(423360, 427680)
2201 Start at  2022-10-29 18:52:00  for  3 days 00:02:00
2202 275 2 (328, 9) 2022-10-22 00:00:00
(423360, 427680)
2202 Start at  2022-10-29 18:52:00  for  3 days 00:02:00
2203 275 3 (328, 9) 2022-10-22 00:00:00
(423360, 427680)
2203 Start at  2022-10-29 18:52:00  for  3 days 00:02:00
2204 275 4 (328, 9) 2022-10-22 00:00:00
(423360, 427680)
2204 Start at  2022-10-29 18:52:00  for  3 days 00:02:00
2205 275 5 (328, 9) 2022-10-22 00:00:00
(423360, 427680)
2205 Start at  2022-10-29 18:52:00  for  3 days 00:02:00
2206 275 6 (328, 9) 2022-10-22 00:00:00
(423360, 427680)
2206 Start at  2022-10-29 18:52:00  for  3 days 00:02:00
2207 275 7 (328, 9) 2022-10-22 00:00:00
(423360, 427680)
2207 Start at  2022-10-29 18:52:00  for  3 days 00:02:00
2208 276 0 (328, 9) 2022-12-20 00:00:00
(508320, 512640)
2209 276 1 (328, 9) 2022-12-20 00:00:00
(508320, 512640)
2210 276 2 (32

(99360, 103680)
2354 Start at  2022-03-14 21:56:00  for  3 days 01:17:00
2355 294 3 (328, 9) 2022-03-11 00:00:00
(99360, 103680)
2355 Start at  2022-03-14 21:56:00  for  3 days 01:17:00
2356 294 4 (328, 9) 2022-03-11 00:00:00
(99360, 103680)
2356 Start at  2022-03-14 21:56:00  for  3 days 01:17:00
2357 294 5 (328, 9) 2022-03-11 00:00:00
(99360, 103680)
2357 Start at  2022-03-14 21:56:00  for  3 days 01:17:00
2358 294 6 (328, 9) 2022-03-11 00:00:00
(99360, 103680)
2358 Start at  2022-03-14 21:56:00  for  3 days 01:17:00
2359 294 7 (328, 9) 2022-03-11 00:00:00
(99360, 103680)
2359 Start at  2022-03-14 21:56:00  for  3 days 01:17:00
2360 295 0 (328, 9) 2022-07-15 00:00:00
(280380, 284700)
2360 Start at  2022-07-21 21:41:00  for  3 days 00:06:00
2361 295 1 (328, 9) 2022-07-15 00:00:00
(280380, 284700)
2361 Start at  2022-07-21 21:41:00  for  3 days 00:06:00
2362 295 2 (328, 9) 2022-07-15 00:00:00
(280380, 284700)
2362 Start at  2022-07-21 21:41:00  for  3 days 00:06:00
2363 295 3 (328, 9) 

(115980, 120300)
2443 Start at  2022-03-27 00:36:00  for  3 days 03:36:00
2444 305 4 (328, 9) 2022-03-23 00:00:00
(115980, 120300)
2444 Start at  2022-03-27 00:36:00  for  3 days 03:36:00
2445 305 5 (328, 9) 2022-03-23 00:00:00
(115980, 120300)
2445 Start at  2022-03-27 00:36:00  for  3 days 03:36:00
2446 305 6 (328, 9) 2022-03-23 00:00:00
(115980, 120300)
2446 Start at  2022-03-27 00:36:00  for  3 days 03:36:00
2447 305 7 (328, 9) 2022-03-23 00:00:00
(115980, 120300)
2447 Start at  2022-03-27 00:36:00  for  3 days 03:36:00
2448 306 0 (328, 9) 2022-04-15 00:00:00
(149760, 154080)
2448 Start at  2022-04-19 17:17:00  for  3 days 03:07:00
2449 306 1 (328, 9) 2022-04-15 00:00:00
(149760, 154080)
2449 Start at  2022-04-19 17:17:00  for  3 days 03:07:00
2450 306 2 (328, 9) 2022-04-15 00:00:00
(149760, 154080)
2450 Start at  2022-04-19 17:17:00  for  3 days 03:07:00
2451 306 3 (328, 9) 2022-04-15 00:00:00
(149760, 154080)
2451 Start at  2022-04-19 17:17:00  for  3 days 03:07:00
2452 306 4 (32

(522720, 527040)
2570 321 2 (328, 9) 2022-12-30 00:00:00
(522720, 527040)
2571 321 3 (328, 9) 2022-12-30 00:00:00
(522720, 527040)
2572 321 4 (328, 9) 2022-12-30 00:00:00
(522720, 527040)
2573 321 5 (328, 9) 2022-12-30 00:00:00
(522720, 527040)
2574 321 6 (328, 9) 2022-12-30 00:00:00
(522720, 527040)
2575 321 7 (328, 9) 2022-12-30 00:00:00
(522720, 527040)
2576 322 0 (328, 9) 2022-01-01 00:00:00
(0, 4320)
2576 Start at  2022-01-01 00:00:00  for  3 days 05:48:00
2577 322 1 (328, 9) 2022-01-01 00:00:00
(0, 4320)
2577 Start at  2022-01-01 00:00:00  for  3 days 05:48:00
2578 322 2 (328, 9) 2022-01-01 00:00:00
(0, 4320)
2578 Start at  2022-01-01 00:00:00  for  3 days 05:48:00
2579 322 3 (328, 9) 2022-01-01 00:00:00
(0, 4320)
2579 Start at  2022-01-01 00:00:00  for  3 days 05:48:00
2580 322 4 (328, 9) 2022-01-01 00:00:00
(0, 4320)
2580 Start at  2022-01-01 00:00:00  for  3 days 05:48:00
2581 322 5 (328, 9) 2022-01-01 00:00:00
(0, 4320)
2581 Start at  2022-01-01 00:00:00  for  3 days 05:48:00

In [163]:
print(train_X.shape, train_Y.shape)
print(valid_X.shape, valid_Y.shape)

(2624, 4320, 53) (2624, 1)
(296, 4320, 53) (296, 1)


In [164]:
with open('train.dat', 'wb') as f:
    np.save(f, train_X)
    np.save(f, train_Y)

In [165]:
with open('valid.dat', 'wb') as f:
    np.save(f, valid_X)
    np.save(f, valid_Y)